In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
except ModuleNotFoundError as e:
    print("not in colab")
    pass
import os
base_dir = "/content/drive/MyDrive/semeval2022"
if not os.path.exists(base_dir):
  !pip install -r requirements.txt
  base_dir = ""
else:
  !pip install -r /content/drive/MyDrive/semeval2022/requirements.txt
  !cp -rf /content/drive/MyDrive/semeval2022/*.py . 
  !cp -rf /content/drive/MyDrive/semeval2022/utils .
  !cp -rf /content/drive/MyDrive/semeval2022/model .

not in colab


In [2]:
from utils.util import get_reader, train_model, create_model, save_model, parse_args, get_tagset, wnut_iob, write_submit_result, load_model, luke_iob
import time

In [3]:
encoder_model = "distilbert-base-uncased"
encoder_model = "roberta-base"
encoder_model = "studio-ousia/luke-base"
track = "EN-English/en"
train_file = os.path.join(base_dir, "training_data/{}_train.conll".format(track))
dev_file = os.path.join(base_dir, "training_data/{}_dev.conll".format(track))
output_dir = os.path.join(base_dir, "{}".format(track), "{}-train".format(encoder_model))
submission_file = os.path.join(base_dir, "submission", "{}.pred.conll".format(track))
iob_tagging = luke_iob
use_crf = True

In [4]:
train_data = get_reader(file_path=train_file, target_vocab=iob_tagging, encoder_model=encoder_model, max_instances=-1, max_length=100)
dev_data = get_reader(file_path=dev_file, target_vocab=iob_tagging, encoder_model=encoder_model, max_instances=-1, max_length=100)

model = create_model(train_data=train_data, dev_data=dev_data, tag_to_id=train_data.get_target_vocab(),
                     dropout_rate=0.1, batch_size=32, stage='fit', lr=2e-5,
                     encoder_model=encoder_model, num_gpus=1, use_crf=use_crf)

trainer = train_model(model=model, out_dir=output_dir, epochs=1, monitor="f1")

# use pytorch lightnings saver here.
out_model_path, best_checkpoint = save_model(trainer=trainer, out_dir=output_dir, model_name=encoder_model, timestamp=time.time())
submission_file = os.path.join(os.path.dirname(out_model_path), "{}.pred.conll".format(track))

model = load_model(best_checkpoint, iob_tagging, use_crf=use_crf)

record_data = write_submit_result(model, dev_data, submission_file)

2021-12-24 15:25:29 - INFO - conll_reader - Reading file training_data/EN-English/en_train.conll
2021-12-24 15:25:31 - INFO - conll_reader - Finished reading 15300 instances from file training_data/EN-English/en_train.conll
2021-12-24 15:25:45 - INFO - conll_reader - Reading file training_data/EN-English/en_dev.conll
2021-12-24 15:25:45 - INFO - conll_reader - Finished reading 800 instances from file training_data/EN-English/en_dev.conll
Some weights of the model checkpoint at studio-ousia/luke-base were not used when initializing LukeForEntitySpanClassification: ['embeddings.position_ids']
- This IS expected if you are initializing LukeForEntitySpanClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LukeForEntitySpanClassification from the checkpoint of a model that you expect to be exactly identical (i

Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNati

AttributeError: 'tuple' object has no attribute 'to'

In [ ]:
record_data.head()

In [ ]:
record_data.groupby(by=['label', 'pred', 'word']).count()

In [ ]:
error_data = record_data[record_data["label"] != record_data["pred"]]
ans = error_data.groupby(by=['label', 'pred']).count().reset_index()
print(ans.to_string())
#ans.sort_values('count')

In [ ]:
error_data.groupby(by=['label', 'pred']).agg('count')